<a href="https://colab.research.google.com/github/MRGotIdeas/twitter_fakenews/blob/main/00_get_fakenews_websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Récupération des sites de fake news

LeMonde met à disposition un fichier json contenant des sites d'information ainsi que leur fiabilité. Ce notebook vise à récupérer cette information, et l'enregistrer au format csv. 

Voir issue #2 : https://github.com/MRGotIdeas/twitter_fakenews

#### Import packages

In [18]:
import pandas as pd
import json
import re

In [24]:
path_data = 'C:/Users/ManonRICHARD/Documents/PFE/twitter-fakenews/PFE/twitter_fakenews/data'

In [30]:
%cd data

C:\Users\ManonRICHARD\Documents\PFE\twitter-fakenews\data


## Import du fichier json

NB : l'enregistrer à partir du site leMonde au format "données brutes" et non au format json

In [33]:
with open(f'fakenews_websites_lemonde.json', 'r') as json_file:
    data = json.loads(json_file.read())

In [34]:
data.keys()

dict_keys(['sites', 'urls'])

**Good to know** : 
- La clé `sites` représente certaines informations comme la fiabilité, un descriptif du site, le nom du site 
- La clé `urls` donne l'ensemble des liens qui appartiennent au site : lien facebook, twitter, youtube, etc

In [ ]:
# un exemple de site 
data['sites']['1393']

In [ ]:
# on cherche l'ensemble des sites qui ont l'identifiant 1393 : buzzarena
[key for key, value in data['urls'].items() if value == 1393]

In [ ]:
# on rajoute la contrainte : on ne veut que les comptes twitter
[key for key, value in data['urls'].items() if value == 1393 and re.search("twitter", key)]

## Formatage des données

On va faire matcher les comptes twitter avec les infos sur les sites

In [94]:
def label_liability(df) : 
    
    df['liability_label'] = df['liability'].map({1:'site parodique', 
                                                 2:'site qui diffuse régulièrement de la fausse information',
                                                 3:'site à la fiabilité douteuse',
                                                 4:'site réputé fiable'})
  
    return df



def format_df_fakenews(json_file) :

    df_sites = pd.DataFrame.from_dict(json_file["sites"], 
                                    orient="index", 
                                    columns=["liability", "description","name","namelow"])\
                                    .reset_index()\
                                    .rename(columns={'index': 'unique_id'})

    df_sites = label_liability(df_sites)
  
    df_url = pd.DataFrame.from_dict(json_file["urls"], orient="index").reset_index()
    df_url.columns = ["path_website", "unique_id"]
    
    df_url['twitter'] =  df_url['path_website'].apply(lambda txt : len(re.findall("twitter.+", txt)) > 0)
    df_url = df_url.loc[df_url['twitter']]
    df_url['unique_id'] = df_url['unique_id'].astype('str')
    
    df_sites = pd.merge(df_sites, df_url, on="unique_id")
    
    return df_sites


In [95]:
df_twitter = format_df_fakenews(data)

In [ ]:
df_twitter.head()

In [97]:
df_twitter['liability_label'].value_counts()

site réputé fiable                                         228
site qui diffuse régulièrement de la fausse information     88
site à la fiabilité douteuse                                59
site parodique                                              50
Name: liability_label, dtype: int64

> Il y a 88 comptes twitter qui diffusent de la fausse information régulièrement \
> Il y a 228 comptes twitter qui sont réputés fiables

--> On va sûrement prendre les sites à la fiabilité douteuse également pour avoir un plus gros effectif de sites de fake news

## Export du dataframe 

In [102]:
df_twitter.to_csv("twitter_accounts_decodex.csv", sep=";", encoding='UTF-8', index=False)